In [ ]:
import tensorflow.compat.v1 as tf
import lucid_kietzmannlab.modelzoo.vision_models as models
from ipywidgets import interact, Dropdown, IntSlider
from lucid_kietzmannlab.utils import interactive_visualization, batch_visualization, plot_images
tf.compat.v1.disable_eager_execution()

In [ ]:
model_checkpoint_dir="/Users/vkapoor/Downloads/models/AlexNet/training_seed_05"
training_seed = 5
model = models.AlexNetv2(model_checkpoint_dir=model_checkpoint_dir, training_seed = training_seed)


In [ ]:
layer_shape_dict = model.layer_shape_dict
def visualize(layer_name, channel):
    interactive_visualization(model,layer_name, channel, layer_shape_dict, scope = 'import')   

def visualize_all():
    # Check if the layer exists in the shape dictionary
    layer_name = current_dropdown_value({'new': layer_dropdown.value})
    image_channel = batch_visualization(model, layer_name, layer_shape_dict, channel_slider, scope = 'import')
    return image_channel              
                              

        
# Create dropdown menu for layer selection
layer_dropdown = Dropdown(options=list(layer_shape_dict.keys()), description='Layer:')

# Create slider for channel selection
channel_slider = IntSlider(min=0, max=0, description='Channel:')
        
        
def update_channel_slider(change):
    layer_name = change.new
    if layer_name in layer_shape_dict:
        
        max_channel = layer_shape_dict[layer_name][-1] - 1
        channel_slider.max = max_channel
        
        
def current_slider_value(*args):
    return channel_slider.value


def current_dropdown_value(change):
    return change['new']


channel_slider.observe(current_slider_value, names='value')
layer_dropdown.observe(current_dropdown_value, names='value')
        

       

In [ ]:
layer_dropdown.observe(update_channel_slider, names='value')

# Create an interactive visualization
interact(visualize, layer_name=layer_dropdown, channel=channel_slider)     

In [ ]:
image_channel = visualize_all()

In [ ]:

if image_channel:
    plot_images(image_channel)   

In [ ]:
layer_shape_dict.values()